In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
nyc_top_zipcodes_all_years = pd.read_csv("./zipcode_data/nyc_top_zipcodes_all_years.csv")
display(nyc_top_zipcodes_all_years.head())

,zipcode,borough,year,count
0,10469,BRONX,2022,122
1,10466,BRONX,2022,119
2,10463,BRONX,2018,105
3,10466,BRONX,2021,99
4,10456,BRONX,2022,98


In [4]:
zillow_home_value = pd.read_csv('../data/clean/zhome_value.csv', low_memory=False)
zillow_home_value["date"] = pd.to_datetime(zillow_home_value["date"]).apply(lambda x: x.year)
zillow_home_value = zillow_home_value.rename(columns={"date": "year", "zip_code": "zipcode"})
display(zillow_home_value.head())

,zipcode,state,city,borough,year,home_value
0,11368,NY,New York,QUEENS,2002,249885.0
1,11385,NY,New York,QUEENS,2002,333404.0
2,11236,NY,New York,BROOKLYN,2002,255687.0
3,11208,NY,New York,BROOKLYN,2002,235676.0
4,10467,NY,New York,BRONX,2002,174696.0


In [5]:
min_year, max_year = nyc_top_zipcodes_all_years["year"].min(), nyc_top_zipcodes_all_years["year"].max()
print("MIN={}\nMAX={}".format(min_year, max_year))

MIN=2015
MAX=2022


In [6]:
zillow_home_value = zillow_home_value[zillow_home_value["year"].between(min_year, max_year, inclusive=True)]
display(zillow_home_value)

/var/folders/2r/0fj15shj34z2mbldl18cp9kc0000gp/T/ipykernel_17194/1115588723.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  zillow_home_value = zillow_home_value[zillow_home_value["year"].between(min_year, max_year, inclusive=True)]


,zipcode,state,city,borough,year,home_value
26486,11368,NY,New York,QUEENS,2015,417755.0
26487,11385,NY,New York,QUEENS,2015,537186.0
26488,11236,NY,New York,BROOKLYN,2015,401310.0
26489,11208,NY,New York,BROOKLYN,2015,396395.0
26490,10467,NY,New York,BRONX,2015,278072.0
...,...,...,...,...,...,...
43058,10464,NY,New York,BRONX,2022,609781.0
43059,10004,NY,New York,MANHATTAN,2022,1081807.0
43060,10006,NY,New York,MANHATTAN,2022,902566.0
43061,11243,NY,New York,BROOKLYN,2022,1426544.0


In [7]:
zillow_home_value.to_csv("zillow_home_value", index = False)

In [8]:
zillow_home_value = zillow_home_value.groupby(["zipcode", "year", "borough"])["home_value"].mean().to_frame().reset_index()
display(zillow_home_value)

,zipcode,year,borough,home_value
0,10001,2015,MANHATTAN,2.280604e+06
1,10001,2016,MANHATTAN,2.369453e+06
2,10001,2017,MANHATTAN,2.271804e+06
3,10001,2018,MANHATTAN,2.351997e+06
4,10001,2019,MANHATTAN,2.380720e+06
...,...,...,...,...
1407,11694,2018,QUEENS,7.692362e+05
1408,11694,2019,QUEENS,8.192713e+05
1409,11694,2020,QUEENS,8.296256e+05
1410,11694,2021,QUEENS,8.971792e+05


In [14]:
zillow_home_value[zillow_home_value["borough"] == "BROOKLYN"]

,zipcode,year,borough,home_value
700,11201,2015,BROOKLYN,1.324822e+06
701,11201,2016,BROOKLYN,1.374819e+06
702,11201,2017,BROOKLYN,1.278541e+06
703,11201,2018,BROOKLYN,1.267816e+06
704,11201,2019,BROOKLYN,1.339921e+06
...,...,...,...,...
1007,11249,2018,BROOKLYN,1.263473e+06
1008,11249,2019,BROOKLYN,1.302043e+06
1009,11249,2020,BROOKLYN,1.287187e+06
1010,11249,2021,BROOKLYN,1.316729e+06


In [15]:
nyc_top_zipcodes_all_years[nyc_top_zipcodes_all_years["borough"] == "BROOKLYN"]

,zipcode,borough,year,count
5,11211,BROOKLYN,2019,6231
6,11211,BROOKLYN,2018,3227
7,11211,BROOKLYN,2022,2730
8,11211,BROOKLYN,2020,2719
9,11211,BROOKLYN,2017,2645


In [9]:
non_intersecting = zillow_home_value.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)

/var/folders/2r/0fj15shj34z2mbldl18cp9kc0000gp/T/ipykernel_17194/1870309356.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  non_intersecting = zillow_home_value.merge(nyc_top_zipcodes_all_years, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)


In [10]:
brooklyn = non_intersecting[non_intersecting["borough"] == "BROOKLYN"]
display(brooklyn)

,zipcode,year,borough,home_value,count
700,11201,2015,BROOKLYN,1.324822e+06,NaN
701,11201,2016,BROOKLYN,1.374819e+06,NaN
702,11201,2017,BROOKLYN,1.278541e+06,NaN
703,11201,2018,BROOKLYN,1.267816e+06,NaN
704,11201,2019,BROOKLYN,1.339921e+06,NaN
...,...,...,...,...,...
1007,11249,2018,BROOKLYN,1.263473e+06,NaN
1008,11249,2019,BROOKLYN,1.302043e+06,NaN
1009,11249,2020,BROOKLYN,1.287187e+06,NaN
1010,11249,2021,BROOKLYN,1.316729e+06,NaN


In [11]:
brooklyn.to_csv("brooklyn_remaining_zipcodes.csv", index=False)

In [12]:
brooklyn_csv = pd.read_csv("brooklyn_remaining_zipcodes.csv")

In [13]:
brooklyn_csv

,zipcode,year,borough,home_value,count
0,11201,2015,BROOKLYN,1.324822e+06,NaN
1,11201,2016,BROOKLYN,1.374819e+06,NaN
2,11201,2017,BROOKLYN,1.278541e+06,NaN
3,11201,2018,BROOKLYN,1.267816e+06,NaN
4,11201,2019,BROOKLYN,1.339921e+06,NaN
...,...,...,...,...,...
302,11249,2018,BROOKLYN,1.263473e+06,NaN
303,11249,2019,BROOKLYN,1.302043e+06,NaN
304,11249,2020,BROOKLYN,1.287187e+06,NaN
305,11249,2021,BROOKLYN,1.316729e+06,NaN
